In [1]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
actions = [
    'spin',
    'come',
    'one_come'
]

data = np.concatenate([
    np.load('dataset/seq_spin_1651820674.npy'),
    np.load('dataset/seq_come_1651820674.npy'),
    np.load('dataset/seq_one_come_1651820674.npy')
], axis=0)

data.shape

(1253, 30, 100)

In [3]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(1253, 30, 99)
(1253,)


In [4]:

from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(1253, 3)

In [5]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(1127, 30, 99) (1127, 3)
(126, 30, 99) (126, 3)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                41984     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 44,163
Trainable params: 44,163
Non-trainable params: 0
_________________________________________________________________


2022-05-06 16:08:46.078029: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 16:08:46.079091: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Train on 1127 samples, validate on 126 samples
Epoch 1/50
1120/1127 [============================>.] - ETA: 0s - loss: 72.6332 - acc: 0.3411
Epoch 00001: val_acc improved from -inf to 0.41270, saving model to models/model.h5
1127/1127 [==============================] - 4s 4ms/sample - loss: 72.3405 - acc: 0.3425 - val_loss: 32.5207 - val_acc: 0.4127
Epoch 2/50
1120/1127 [============================>.] - ETA: 0s - loss: 24.3918 - acc: 0.4759
Epoch 00002: val_acc improved from 0.41270 to 0.48413, saving model to models/model.h5
1127/1127 [==============================] - 3s 2ms/sample - loss: 24.2984 - acc: 0.4765 - val_loss: 18.1163 - val_acc: 0.4841
Epoch 3/50
1120/1127 [============================>.] - ETA: 0s - loss: 7.1631 - acc: 0.6607
Epoch 00003: val_acc improved from 0.48413 to 0.69841, saving model to models/model.h5
1127/1127 [==============================] - 3s 2ms/sample - loss: 7.1187 - acc: 0.6628 - val_loss: 4.6909 - val_acc: 0.6984
Epoch 4/50
1120/1127 [=============

1120/1127 [============================>.] - ETA: 0s - loss: 0.2426 - acc: 0.9357
Epoch 00030: val_acc did not improve from 0.96825
1127/1127 [==============================] - 8s 7ms/sample - loss: 0.2412 - acc: 0.9361 - val_loss: 0.3958 - val_acc: 0.8730
Epoch 31/50
1120/1127 [============================>.] - ETA: 0s - loss: 0.1752 - acc: 0.9527
Epoch 00031: val_acc did not improve from 0.96825
1127/1127 [==============================] - 5s 5ms/sample - loss: 0.1745 - acc: 0.9530 - val_loss: 0.0647 - val_acc: 0.9683
Epoch 32/50
1120/1127 [============================>.] - ETA: 0s - loss: 0.1414 - acc: 0.9554
Epoch 00032: val_acc improved from 0.96825 to 0.97619, saving model to models/model.h5
1127/1127 [==============================] - 4s 3ms/sample - loss: 0.1418 - acc: 0.9547 - val_loss: 0.0753 - val_acc: 0.9762
Epoch 33/50
1120/1127 [============================>.] - ETA: 0s - loss: 0.1473 - acc: 0.9625
Epoch 00033: val_acc improved from 0.97619 to 0.98413, saving model to mod

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))